In [33]:
from tpk4170.visualization import Ur5Visualizer
from dataclasses import dataclass
import modern_robotics as mr
import numpy as np
import sympy as sp

np.set_printoptions(suppress=True)

PI_HALF = np.pi / 2
PI = np.pi


In [34]:
@dataclass
class DHLink:
    a: float
    alpha: float
    d: float
    theta: float

    def matrix(self) -> np.array:
        ct = np.cos(self.theta)
        st = np.sin(self.theta)
        ca = np.cos(self.alpha)
        sa = np.sin(self.alpha)
        return np.array(
            [
                [ct, -st * ca, st * sa, self.a * ct],
                [st, ct * ca, -ct * sa, self.a * st],
                [0.0, sa, ca, self.d],
                [0.0, 0.0, 0.0, 1.0],
            ]
        )


In [35]:
class KR6:


    def __init__(self):

        self.d1, self.d2, self.d3, self.d4 = 0.400, 0.455, 0.420, 0.080 #distansene mellom jointsene som ikke er 0
        self.a1, self.a2 = 0.025, 0.035 #offsetsene 

        self._M = np.array(
            [
                [0,  0,  1, self.a1+self.d2+self.d3+self.d4],
                [0, -1,  0,                               0],
                [1,  0,  0,                 self.a2+self.d1],
                [0,  0,  0,                               1]
            ]
        )

        self._Slist = np.array(
            [
                [0, 0,-1,                  0,                 0,                       0],
                [0, 1, 0,           -self.d1,                 0,                 self.a1],
                [0, 1, 0,           -self.d1,                 0,         self.a1+self.d2],
                [-1,0, 0,                  0,-(self.a2+self.d1),                       0],
                [0, 1, 0, -(self.d1+self.a2),                 0, self.a1+self.d2+self.d3],
                [-1,0, 0,                  0,-(self.a2+self.d1),                       0]
            ]
        ).T
        
        self.theta = np.array([ [0],        # from space frame to joint 1
                                [0],        # from joint 1 to joint 2
                                [0],        # from joint 2 to joint 3
                                [-PI_HALF], # from joint 3 to joint 4
                                [0],        # from joint 4 to joint 5
                                [0],        # from joint 5 to joint 6
                                [0]])       # from joint 6 to body frame

        self.alpha = np.array([[PI],
                        [PI_HALF],
                        [0],
                        [PI_HALF],
                        [-PI_HALF],
                        [PI_HALF],
                        [PI]])

        self.d = np.array([[0],
                    [-self.d1],
                    [0],
                    [0],
                    [-self.d3],
                    [0],
                    [-self.d4]])

        self.a = np.array([[0],
                    [self.a1],
                    [self.d2],
                    [self.a2],
                    [0],
                    [0],
                    [0]])

        self.dhList = np.concatenate((self.a, self.alpha, self.d, self.theta), axis=1)

        self._Blist = mr.Adjoint(np.linalg.inv(self._M)) @ self._Slist

        self.linkList = [] # Contains all the DHlinks for the robot
        for i in range(len(self.dhList)):
            self.linkList.append(DHLink(*self.dhList[i]))

        self.fk_dh_zero = self.linkList[0].matrix() # DH zero configuration
        self.Tlist = [self.linkList[0].matrix()]    # Contains all the transformation matrices

        for i in range(1, len(self.dhList)):
            self.fk_dh_zero *= self.linkList[i].matrix()
            self.Tlist.append(self.Tlist[i-1] @ self.linkList[i].matrix())

    def fKinS(self, theta): #Returnerer T_SB, altså transformasjonsmatrisa fra space til end-effector
        return mr.FKinSpace(self._M, self._Slist, theta)


In [36]:
kr6 = KR6()

In [37]:
from tpk4170.visualization import Viewer
from tpk4170.models import Grid, Axes, Ball
from pythreejs import Object3D
from transformations import quaternion_from_matrix
from pythreejs import ConeGeometry
from pythreejs import Preview

In [38]:
class DHFrame(Axes):
    def __init__(self, trf):
        Axes.__init__(self, 0.1)
        self.quaternion = np.roll(quaternion_from_matrix(trf), -1).tolist()
        self.position = (trf[:3, 3]).tolist()

In [39]:
viewer = Viewer(background="white")
viewer.add(Grid())

viewer.add(Axes(0.1))
viewer.add(DHFrame(kr6.Tlist[0]))
viewer.add(DHFrame(kr6.Tlist[1]))
viewer.add(DHFrame(kr6.Tlist[2]))
viewer.add(DHFrame(kr6.Tlist[3]))
viewer.add(DHFrame(kr6.Tlist[4]))
viewer.add(DHFrame(kr6.Tlist[5]))
viewer.add(DHFrame(kr6.Tlist[6]))

C:\Users\Bendik\.conda\envs\tpk4170\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…